In [ ]:
# This version uses only power values and FAP as parameters to predictors, the earlier version used power, FAP and frequencies

In [1]:
import lightkurve as lk
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from astropy.timeseries import LombScargle
from sklearn.preprocessing import StandardScaler

In [2]:
file = pd.read_csv('Rafael_Sample.csv', sep=';')
data_table = pd.DataFrame(columns=['p0', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'fap', 'Class'])

for i in range(len(file)):
    search = lk.search_lightcurve(f'TIC {file.iloc[i, 0]}', author='SPOC', exptime=120)
    for j in range(len(search)):
        sector = search.mission[j][12:]
        lc = search[j].download(download_dir='E:/lightkurve_fits/').normalize().remove_outliers(sigma=3.5)
        pg = lc.to_periodogram(maximum_frequency=50)
        
        # This block lists the power values and then order from the maximum value
        df_pg_tab = pg.to_table().to_pandas()
        table = df_pg_tab.filter(['power'])
        df_ord_power = pd.DataFrame(data=sorted(table.power, reverse=True), columns=['ord_power'])
        
        # Calculating the False Alarm Probability (FAP)
        time = lc.time
        flux = lc.flux
        fap = LombScargle(time, flux).false_alarm_probability(pg.max_power)*100
        
        # Inserting the data (power values, FAP and Classes) in the data_table
        temp_tab = pd.DataFrame(data={'p0': df_ord_power.ord_power[0], 'p1':df_ord_power.ord_power[1], 'p2':df_ord_power.ord_power[2], 'p3':df_ord_power.ord_power[3], 'p4':df_ord_power.ord_power[4], 'p5':df_ord_power.ord_power[5], 'p6':df_ord_power.ord_power[6], 'p7':df_ord_power.ord_power[7], 'p8':df_ord_power.ord_power[8], 'p9':df_ord_power.ord_power[9], 'fap':fap.value, 'Class':file.iloc[i, 1]}, index=[f'{file.iloc[i, 0]}_{sector}'])
        data_table = pd.concat([data_table, temp_tab], axis=0)
        
data_table = data_table.rename_axis('TIC_Sect').reset_index()
#data_table
print('DONE!')

C:\Users\jcfis\anaconda3\Lib\site-packages\lightkurve\lightcurve.py:1131: LightkurveWarning: The light curve appears to be zero-centered (median=-5.34e+00 electron / s +/- 1.49e+01 electron / s); `normalize()` will divide the light curve by a value close to zero, which is probably not what you want.
  warnings.warn(
C:\Users\jcfis\anaconda3\Lib\site-packages\lightkurve\lightcurve.py:1142: LightkurveWarning: The light curve has a negative median flux (-5.34e+00 electron / s); `normalize()` will therefore divide by a negative number and invert the light curve, which is probablynot what you want
  warnings.warn(
C:\Users\jcfis\anaconda3\Lib\site-packages\lightkurve\lightcurve.py:1131: LightkurveWarning: The light curve appears to be zero-centered (median=-1.00e+00 electron / s +/- 8.27e+00 electron / s); `normalize()` will divide the light curve by a value close to zero, which is probably not what you want.
  warnings.warn(
C:\Users\jcfis\anaconda3\Lib\site-packages\lightkurve\lightcurve.

C:\Users\jcfis\anaconda3\Lib\site-packages\lightkurve\lightcurve.py:1131: LightkurveWarning: The light curve appears to be zero-centered (median=4.88e+00 electron / s +/- 1.34e+01 electron / s); `normalize()` will divide the light curve by a value close to zero, which is probably not what you want.
  warnings.warn(
C:\Users\jcfis\anaconda3\Lib\site-packages\lightkurve\lightcurve.py:1131: LightkurveWarning: The light curve appears to be zero-centered (median=1.20e+00 electron / s +/- 7.48e+00 electron / s); `normalize()` will divide the light curve by a value close to zero, which is probably not what you want.
  warnings.warn(
C:\Users\jcfis\anaconda3\Lib\site-packages\astropy\timeseries\periodograms\lombscargle\_statistics.py:140: RuntimeWarning: invalid value encountered in scalar power
  return (1 - z) ** (0.5 * Nk)
C:\Users\jcfis\anaconda3\Lib\site-packages\lightkurve\lightcurve.py:1131: LightkurveWarning: The light curve appears to be zero-centered (median=1.11e+00 electron / s +/-

DONE!


In [3]:
data_table.shape

(3969, 13)

# Dividing between predictors and class

In [4]:
x_param = data_table.iloc[:,1:12].values
x_param

array([[1.79985823e-04, 1.69132726e-04, 1.58353399e-04, ...,
        1.13186383e-04, 1.12843173e-04, 1.00000000e+02],
       [2.19509067e-04, 2.13647452e-04, 1.92579572e-04, ...,
        1.33448385e-04, 1.20786836e-04, 1.00000000e+02],
       [1.98871520e-04, 1.91388494e-04, 1.78379292e-04, ...,
        1.50431403e-04, 1.50349740e-04, 1.00000000e+02],
       ...,
       [1.28656969e-02, 1.24568537e-02, 1.14382554e-02, ...,
        4.96552945e-03, 4.72904959e-03, 1.20502053e-42],
       [7.72558167e-03, 7.19056966e-03, 7.09960231e-03, ...,
        3.24531668e-03, 2.86948501e-03, 5.94441325e-20],
       [8.20182033e-03, 8.05936936e-03, 7.07224529e-03, ...,
        3.89585618e-03, 3.53043257e-03, 7.39884483e-18]])

In [5]:
y_param = data_table.iloc[:,12].values
y_param

array(['Noisy', 'Noisy', 'Noisy', ..., 'Var', 'Var', 'Var'], dtype=object)

### ~ Attribute scaling

In [6]:
scaler = StandardScaler()
x_param = scaler.fit_transform(x_param)
x_param

array([[-0.09646576, -0.0960075 , -0.09715604, ..., -0.09714572,
        -0.09651145,  0.84989482],
       [-0.09582296, -0.09526503, -0.09653901, ..., -0.09656141,
        -0.09627526,  0.84989482],
       [-0.0961586 , -0.09563629, -0.09679502, ..., -0.09607165,
        -0.09539627,  0.84989482],
       ...,
       [ 0.10985303,  0.10894182,  0.10619831, ...,  0.04278615,
         0.04074277, -1.24452966],
       [ 0.02625485,  0.02110443,  0.02798096, ..., -0.00682135,
        -0.01454789, -1.24452966],
       [ 0.03400034,  0.03559531,  0.02748777, ...,  0.01193891,
         0.00510414, -1.24452966]])

### ~ Dividing between training and test sample

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x_param, y_param, test_size=0.15)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(3373, 11) (3373,)
(596, 11) (596,)


# Saving data

In [8]:
with open('Rafael_data.pkl', 'wb') as file:
    pickle.dump([x_train, y_train, x_test, y_test], file)